# Import Brightway libraries

In [ ]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc
from functools import partial

# Restore ecoinvent 3.8 project

In [ ]:
import bw2io as bi
bi.restore_project_directory("/srv/data/projects/ecoinvent38.tar.gz")

In [ ]:
bd.projects.set_current("bw2_class_import_example")

In [ ]:
bd.databases

# Provide method metadata

In [ ]:
DESCRIPTION=("A midpoint-level mineral resource impact assessment method matching this perspective is proposed, "
             "called the crustal scarcity indicator (CSI), with characterization factors called crustal scarcity "
             "potentials (CSPs) measured as kg silicon equivalents per kg element. They are based on crustal "
             "concentrations, which have been suggested to correlate with several important resource metrics "
             "(reserves, reserve base, reserves plus cumulative production, and ore deposits), thereby constituting "
             "proxies for long-term global elemental scarcity.")

You can download the file `CSI_CF.xlsx` from [github](https://github.com/brightway-lca/brightway2/raw/master/notebooks/CSI_CF.xlsx) if you need it (if you cloned the Git repo it is there already).

In [ ]:
ei = bi.ExcelLCIAImporter(
    filepath="files/CSI_CF.xlsx", 
    name=("Crustal Scarcity Index", "1.0"), 
    description=DESCRIPTION, 
    unit="kilogram Silicon equivalent",
    # Can add any additional metadata
    article_url="https://link.springer.com/article/10.1007%2Fs11367-020-01781-1",
    doi="https://doi.org/10.1007/s11367-020-01781-1",
    authors="Rickard Arvidsson, Maria Ljunggren Söderman, Björn A. Sandén, Anders Nordelöf, Hampus André, Anne-Marie Tillman",
)

# Data cleanup

## Step 1: Remove empty lines

Because of the way the Excel works, lines you touch but are still empty can be considered part of the worksheet. We remove these.

In [ ]:
[cf for cf in ei.data[0]['exchanges'] if not cf['Flow']]

In [ ]:
def drop_empty_lines(data):
    for method in data:
        method['exchanges'] = [obj for obj in method['exchanges'] if obj['Flow']]
    return data    

In [ ]:
ei.apply_strategy(drop_empty_lines)

## Step 2: Change `Flow` to `name`, 'Chracterization Factor' to 'amount'

We need to match the `name` in the CF to the `name` in the biosphere database.

In [ ]:
def rename_flow(data):
    for method in data:
        for cf in method['exchanges']:
            cf['name'] = cf.pop('Flow')
            cf['amount'] = cf.pop('Chracterization Factor')
    return data

In [ ]:
ei.apply_strategy(rename_flow)

## Step 3: Change category to ecoinvent standard

CSI input data uses `Resource`, but ecoinvent data uses `natural resource`.

In [ ]:
def rename_resource_category(data):
    for method in data:
        for cf in method['exchanges']:
            if cf['Category'] == 'Resource':
                cf['Category'] = 'natural resource'
    return data

In [ ]:
ei.apply_strategy(rename_resource_category)

## Step 4: Create tuple of `categories`

Brightway support a [tuple](https://en.wikipedia.org/wiki/Tuple) of categories of arbitrary length, while CSI uses length 1 or 2. Length 1 uses `undefined` as a dummy parameter. We convert the fields `Category` and `Sub category` to a tuple of uniform style, without dummy parameters.

In [ ]:
def create_categories_as_tuple(data):
    for method in data:
        for cf in method['exchanges']:
            if cf['Sub category'] == 'unspecified':
                cf['categories'] = (cf['Category'],)
            else:
                cf['categories'] = (cf['Category'], cf['Sub category'])
    return data

In [ ]:
ei.apply_strategy(create_categories_as_tuple)

# Run the normal strategies

This includes linking against the ecoinvent biosphere database.

In [ ]:
ei.apply_strategies()

Check our results.

In [ ]:
ei.statistics()

Not great. As with all IO operations, we now enter an iterative improvement period...

# Iterative improvement

## Step 1: Name discrepancies

First, we notice that there are some name discrepancies. Namely, CSI provides reasonable names:

In [ ]:
[x for x in ei.data[0]['exchanges'] if x['name'] == 'Copper, 0.59% in sulfide, Cu 0.22% and Mo 8.2E-3% in crude ore']

But ecoinvent does something weird:

In [ ]:
[x 
 for x in bd.Database(bd.config.biosphere) 
 if 'Copper, 0.59% in sulfide, Cu 0.22% and Mo 8.2E-3% in crude ore' in x['name'] 
 and x['categories'][0] == 'natural resource']

It is subtle, but for some reason ecoinvent adds the suffix **`, in ground`**, even though we know that as the category is `in ground`.

We also notice a mismatch here: The `categories` don't match. CSI doesn't have a CF for `('natural resource', 'in ground')`, while ecoinvent doesn't have a flow for `('natural resource',)`.

We will need two strategies to fix these two issues.

In [ ]:
def add_in_ground_if_helpful(data):
    biosphere = {o['name'] 
                 for o in bd.Database(bd.config.biosphere) 
                 if o['categories'][0] == 'natural resource'}
    
    for method in data:
        for cf in method['exchanges']:
            if not cf.get('input'):  # Skip already linked CFs
                if cf['name'] not in biosphere and cf['name'] + ", in ground" in biosphere:
                    cf['name'] += ", in ground"
    
    return data

In [ ]:
ei.apply_strategy(add_in_ground_if_helpful)

In [ ]:
def change_categories_if_helpful(data):
    biosphere = {(o['name'], tuple(o['categories'])) for o in bd.Database(bd.config.biosphere)}
    
    for method in data:
        for cf in method['exchanges']:
            if cf.get('input'):  # Skip already linked CFs
                continue
            if (
                    (cf['name'], cf['categories']) not in biosphere 
                    and (cf['name'], cf['categories'] + ("in ground",)) in biosphere
                ):
                cf['categories'] = cf['categories'] + ("in ground",)
    
    return data    

In [ ]:
ei.apply_strategy(change_categories_if_helpful)

Try linking with our changes.

In [ ]:
ei.apply_strategy(partial(
    bi.strategies.link_iterable_by_fields,
    other=bd.Database(bd.config.biosphere),
    kind="biosphere",
    fields=("name", "categories"),
))

In [ ]:
ei.statistics()

## Step 2: Pain

Ecoinvent has the following flows for copper:

* Copper, 1.18% in sulfide, Cu 0.39% and Mo 8.2E-3% in crude ore, in ground
* Copper, 1.25% in sulfide, Cu 0.24% and Zn 0,1% in crude ore, in ground
* Copper, 1.42% in sulfide, Cu 0.81% and Mo 8.2E-3% in crude ore, in ground

etc. Oh, and also:

* Cu, Cu 5.2E-2%, Pt 4.8E-4%, Pd 2.0E-4%, Rh 2.4E-5%, Ni 3.7E-2% in ore, in ground

<img src="files/consistency.png">

Let's see what the CST developers have and don't have:

In [ ]:
ei.write_excel("CSI")

This file is also available [here](files/lcia-matching-CSI.xlsx).

After checking these against the list of biosphere flows in ecoinvent 3.8, it looks like the CSI developers got everything! The missing flows only exist in SimaPro for compatibiltiy with other databases.

BTW, if you want to see a more reasonable list of flows, check out https://github.com/USEPA/Federal-LCA-Commons-Elementary-Flow-List.

## Step 3: Drop the missing CFs

These are for flows not present in the "clean" version of ecoinvent, and so won't matter for our results.

In [ ]:
ei.drop_unlinked(True)

In [ ]:
ei.statistics()

## Step 4: Check to make sure we don't have duplicates

Due to the way the [scipy sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) matrix constructors work, duplicate CFs would be summed during the construction of the characterization matrix. This would be bad :)

In [ ]:
len(ei.data[0]['exchanges']), len({tuple(o['input']) for o in ei.data[0]['exchanges']})

C'est pas bon... but at least easy to fix.

In [ ]:
unique_exchanges = {tuple(o['input']): o for o in ei.data[0]['exchanges']}
ei.data[0]['exchanges'] = list(unique_exchanges.values())

In [ ]:
ei.write_methods(overwrite=True)

# Check against reference values

In [ ]:
steel = next(x for x in bd.Database("ecoinvent 3.8 cutoff") if x['name'] == 'market for steel, low-alloyed')
steel

In [ ]:
REFERENCE = 433.46247

In [ ]:
lca = bc.LCA({steel: 1}, ("Crustal Scarcity Index", "1.0"))
lca.lci()
lca.lcia()
print(lca.score / REFERENCE)

In [ ]:
copper = next(x for x in bd.Database("ecoinvent 3.8 cutoff") if x['name'] == 'market for copper')  # carbonate, sulfate, anode
copper

In [ ]:
REFERENCE = 23868.3

In [ ]:
lca = bc.LCA({copper: 1}, ("Crustal Scarcity Index", "1.0"))
lca.lci()
lca.lcia()
print(lca.score / REFERENCE)

In [ ]:
nickel = next(x for x in bd.Database("ecoinvent 3.8 cutoff") if x['name'] == 'market for nickel 95%')  # sulfate?
nickel

In [ ]:
REFERENCE = 4.78E+04

In [ ]:
lca = bc.LCA({nickel: 1}, ("Crustal Scarcity Index", "1.0"))
lca.lci()
lca.lcia()
print(lca.score / REFERENCE)

Pretty good. The details on the [SimaPro matrix algorithm](https://pre-sustainability.com/files/2014/05/The_Improved_SimaPro_8_Calculation_Engine_Whitepaper_A4.pdf) are not transparent, but I think they do something iterative. We use [pardiso](https://www.pardiso-project.org/), which in my judgment is more reliable, but I don't have actual evidence either way.